In [1]:
from pyscal.core import System
from pyscal.crystal_structures import Structure
from pyscal.operations.symmetry import Symmetry
import pyscal_rdf.properties as prp
from pyscal_rdf.json_io import write_file
import numpy as np
import graphviz

Update hedgedoc [here](https://iffmd.fz-juelich.de/BRIkKGLlQduS8dA0sXKoZw)

In [2]:
def convert_to_dict(sys):
    """
    Convert a pyscal System object to data dictionary
    
    Parameters
    ----------
    sys: pyscal System
        input system
    
    Returns
    -------
    info: dict
        dict with parsed information
    """
    info = {}
    
    
    #Simulation cell properties
    #--------------------------------------------------------------    
    info["ChemicalCompositionElement"] = list(sys.composition.keys())
    info["ChemicalCompositionRatio"] = [val for key, val in sys.composition.items()]    
    info["CellVolume"] = sys.volume
    info["NumberOfAtoms"] = sys.natoms
    
    box_dims = sys.box_dimensions
    info["SimulationCellLengthX"] = box_dims[0]
    info["SimulationCellLengthY"] = box_dims[1]
    info["SimulationCellLengthZ"] = box_dims[2]

    info["SimulationCellVectorA"] = sys.box[0]
    info["SimulationCellVectorB"] = sys.box[1]
    info["SimulationCellVectorC"] = sys.box[2]
    
    info["SimulationCellAngleAlpha"] = prp.get_angle(sys.box[0], sys.box[1])
    info["SimulationCellAngleBeta"] = prp.get_angle(sys.box[1], sys.box[2])
    info["SimulationCellAngleGamma"] = prp.get_angle(sys.box[2], sys.box[0])    
    
    #Atom properties
    #--------------------------------------------------
    if sys.atoms.species[0] is not None:
        info["Element"] = sys.atoms.species
    else:
        info["Element"] = sys.atoms.types
    info["Coordination"] = prp.get_coordination(sys)
    info["Positions"] = sys.atoms.positions
    
    #unit cell things
    #--------------------------------------------------
    #Unit cell properties
    info["LatticeParameter"] = sys.atoms._lattice_constant
    
    #do only for unit cell
    if sys._structure_dict is None:
        pass
        #maybe try to find space group with warnings!
    else:
        symbol, number = prp.get_space_group(sys)
        info["SpaceGroupSymbol"] = symbol
        info["SpaceGroupNumber"] = number
        info["CrystalStructureName"] = sys.atoms._lattice 
        info["BravaisLattice"] = prp.get_bravais_lattice(sys)
        info["BasisPositions"] = sys._structure_dict['positions']
        info["BasisOccupancy"] = prp.get_basis(sys)
        info["LatticeVectors"] = prp.get_lattice_vector(sys)
        
    return info

## Creating structure from pyscal

In [3]:
struct = Structure()

In [4]:
sys = struct.lattice.l12(element=['Al', 'Ni'], 
                         lattice_constant=3)

['Al', 'Ni']


In [5]:
sysdict = convert_to_dict(sys)

In [6]:
write_file("structure.json", sysdict)

## RDFLIB Mapping

Assuming one structure dictionary

In [7]:
from rdflib import Graph, Literal, Namespace, XSD, RDF, RDFS, BNode, URIRef, FOAF, SKOS

In [8]:
CMSO = Namespace("https://purls.helmholtz-metadaten.de/cmso/")

In [9]:
g = Graph()
sample = BNode('Sample')
g.add((sample, RDF.type, CMSO.AtomicScaleSample))
#g.add((sample, SKOS.altLabel, Literal("A")))

material = BNode('Material')
g.add((sample, CMSO.hasMaterial, material))
g.add((material, RDF.type, CMSO.CrystallineMaterial))
#g.add(())

<Graph identifier=Nfc3759375ac64870864fe4a78cf71167 (<class 'rdflib.graph.Graph'>)>

In [ ]:
Coordinate_Vector = URIRef("https://w3id.org/mdo/structure/CoordinateVector")
Lattice = URIRef("https://purls.helmholtz-metadaten.de/disos/cso#Lattice")
Structure

In [ ]:

g.add((Coordinate_Vector, MDO.X_axisCoordinate, Literal(data["X_AxisCoordinate"])))
g.add((Coordinate_Vector, MDO.Y_axisCoordinate, Literal(data["Y_AxisCoordinate"])))
g.add((Coordinate_Vector, MDO.Z_axisCoordinate, Literal(data["Z_AxisCoordinate"])))

In [ ]:
with open("dump-ld.json", "w") as fout:
    fout.write(g.serialize(format="json-ld"))

In [ ]:
print(g.serialize(format="json-ld"))

- Map the terms (see above)
- Terms for type annotations
- Use BNode to add multiple structures
- Add visualisation
- Space group calculation
- try saving to database
- Query the graph (range queries possible?)
- Make a package out of it
- Extend....

## Examples

### POSCAR

In [4]:
sys = System("al_data/Al.poscar", format="poscar")

In [5]:
convert_to_dict(sys)
write_file("dump", convert_to_dict(sys))

In [6]:
sys.composition

{'Al': 1.0}

### LAMMPS - dump

In [7]:
sys = System("al_data/Al.dump")

In [8]:
convert_to_dict(sys)
write_file("dump2", convert_to_dict(sys))

In [9]:
sys.composition

{1: 1.0}

### CIF/ ASE

In [7]:
from ase.io import read

In [8]:
aseobj = read("al_data/Al.cif", format="cif")

In [9]:
sys = System(aseobj, format="ase")

In [10]:
convert_to_dict(sys)
write_file("dump3", convert_to_dict(sys))

In [11]:
data = convert_to_dict(sys)